Program 2 - For a given set of training examples, implement and demonstrate the Candidate Elimination Algorithm to output a description of the set of all hypothesis consistent with training examples

1:Procedure CandidateEliminationLearner(X,Y,E,H)  
2:&nbsp;&nbsp;Inputs  
3:&nbsp;&nbsp;&nbsp;&nbsp;X: set of input features, X={X1,...,Xn}  
4:&nbsp;&nbsp;&nbsp;&nbsp;Y: target feature  
5:&nbsp;&nbsp;&nbsp;&nbsp;E: set of examples from which to learn  
6:&nbsp;&nbsp;&nbsp;&nbsp;H: hypothesis space  
7:&nbsp;&nbsp;Output  
8:&nbsp;&nbsp;&nbsp;&nbsp;general boundary G⊆H  
9:&nbsp;&nbsp;&nbsp;&nbsp;specific boundary S⊆H consistent with E  
10:&nbsp;&nbsp;Local  
11:&nbsp;&nbsp;&nbsp;&nbsp;G: set of hypotheses in H  
12:&nbsp;&nbsp;&nbsp;&nbsp;S: set of hypotheses in H  
13:&nbsp;&nbsp;Let G={true}, S={false};  
14:&nbsp;&nbsp;&nbsp;for each e∈E do  
15:&nbsp;&nbsp;&nbsp;&nbsp;if ( e is a positive example) then  
16:&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Elements of G that classify e as negative are removed from G;
17:&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Each element s of S that classifies e as negative is removed and replaced by the
18:&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;minimal generalizations of s that classify e as positive and are less general than
19:&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;some member of G;  
20:&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Non-maximal hypotheses are removed from S;  
21:&nbsp;&nbsp;&nbsp;&nbsp;else  
22:&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Elements of S that classify e as positive are removed from S;  
23:&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Each element g of G that classifies e as positive is removed and replaced by the   
24:&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;minimal specializations of g that classifies e as negative and are more general than  
25:&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;some member of S.  
26:&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Non-minimal hypotheses are removed from G.  

### Importing the required Libraries  

csv module is imported for handling the data set

In [24]:
import random
import pandas as pd
import csv

### Displaying the data set

In [25]:
df = pd.read_csv('CandidateElimination.csv')
with open('CandidateElimination.csv') as csvFile:
    examples = [tuple(line) for line in csv.reader(csvFile)]
print(df)

    Size Color     Shape Label
0    big   red    circle    No
1  small   red  triangle    No
2  small  red     circle   Yes
3    big  blue    circle    No
4  small  blue    circle   Yes


In [26]:
def get_domains(examples):
    #creates an empty set for the number of columns
    d = [set() for i in examples[0]]
       
    for x in examples:

        for i,xi in enumerate(x):
            d[i].add(xi)
    return [list(sorted(x)) for x in d]

get_domains(examples)

[['Size', 'big', 'small'],
 ['Color', 'blue', 'red', 'red '],
 ['Shape', 'circle', 'triangle'],
 ['Label', 'No', 'Yes']]

In [27]:
def generalize_S(x, G, S):
    S_prev = list(S)
    
    for s in S_prev:
        if (not fulfills(x,S)):
            S.remove(s)
            S_plus = min_generalization(S,x)
    
            S.update([h for h in S_plus if any(more_general(g,h) for g in G)])
            S.difference_update([h for h in S if any([more_general(h,h1) for h1 in S if h!=h1])])
    
    return S

In [28]:
def specialize_G(x,domains, G,S):
    G_prev = list(G)
    
    for g in G_prev:
        if(fulfills(x,g)):
            G.remove(g)
            G_minus = min_specialization(g,domains,x)
        
            G.update(h for h in G_minus if any([more_general(h,s) for s in S]))
    
            G.difference_update([h for h in G if any([more_general(g1,h) for g1 in G if h!=g1])])
    return G        

In [29]:
def fulfills(example, hypothesis):
    return more_general(hypothesis,example)


In [30]:
def min_generalization(h,x):
    h_new = list(h)
    
    for i in range(len(h)):
        if(not fulfills(x[i:i+1],h[i:i+1])):
            h_new = '?' if h[i] != '0' else x[i]
    return [tuple(h_new)]


In [31]:
def min_specialization(h,domains,x):
    results = []
    
    for i in range(len(h)):
        if h[i] == '?':
            for val in domains[i]:
                if(x[i]!= val):
                    h_new = h[:i]+(val, )+h[i+1:]
                    results.append(h_new)
        elif(h[i]!='/0'):
            h_new = h[:i]+('0', )+h[i+1:]
            results.append(h_new)
    return results


In [32]:
def more_general(h1,h2):
    more_general_parts = []
    
    for x,y in zip(h1,h2):
        mg = x =='?' or (x!='0' and (x==y or y=='0'))
        more_general_parts.append(mg)
    return all(more_general_parts)


In [33]:
def g_0(n):
    return ('?',)*n


In [34]:
def s_0(n):
    return ('0',)*n


In [35]:
def candidate_elimination(examples):
    domains = get_domains(examples)[:-1]
    G = set([g_0(len(domains))])
    S = set([s_0(len(domains))])
    
    i=0
    print('\n G[{0}]:'.format(i),G)
    print('\n S[{0}]'.format(i),S)
    
    for xcx in examples:
        i = i+1
        
        
        x,cx = xcx[:-1], xcx[-1]
    
        if (cx=='Yes'):
            G = {g for g in G if fulfills(x,g)}
            S = generalize_S(x,G,S)
            
        else:
            S = {s for s in S if not fulfills(x,S)}
            G = specialize_G(x, domains,G,S)
candidate_elimination(examples)


 G[0]: {('?', '?', '?')}

 S[0] {('0', '0', '0')}
